<a href="https://colab.research.google.com/github/PatrickStevensCGU/IST322_Summer2021_GroupProject/blob/main/IST322_GroupProject_YelpScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time 
import random
import urllib

In [2]:
# this you need to do everytime if you do not choose "permantly mount" option. 
from google.colab import drive
drive.mount('/content/drive')

# Mount the local drive
%cd /content/drive/MyDrive/
if not os.path.exists('Mydata'):
  os.mkdir('Mydata')

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
%cd /content/drive/MyDrive/Mydata
#df = pd.read_excel('SBRC_Doctor_Reviews.csv', names=['review_rating','date','review','restaurant','city','state','postal_code','restaurant_rating'], header=None, index_col=False)
original_doctor_df = pd.read_excel('SBRC_Doctor_Reviews.xlsx', index_col=False)
print(f'Total review count: {len(original_doctor_df)}')

/content/drive/MyDrive/Mydata
Total review count: 2607


In [4]:
original_doctor_df.head()

,Business ID,Name,Business Category,URL,snippet text,Address,City,State code,zip code,clamed,closed,latitude,longitude,phone
0,rancho-wellness-rancho-cucamonga-7,Rancho Wellness,"Family Practicefamilydr, Weight Loss Centerswe...",https://www.yelp.com/biz/rancho-wellness-ranch...,First off let me say that it takes A LOT for m...,8231 Rochester AveRancho WellnessRancho Cucamo...,Rancho Cucamonga,CA,91730.0,1,0,34.103970,-117.544580,9.094838e+09
1,san-bernardino-medical-group-san-bernardino,San Bernardino Medical Group,"Family Practicefamilydr, Internal Medicineinte...",https://www.yelp.com/biz/san-bernardino-medica...,I have been coming to SBMG for over ten years ...,"1700 N Waterman AveSan Bernardino, CA 92404",San Bernardino,CA,92404.0,1,0,34.130330,-117.279740,9.098839e+09
2,st-bernardine-medical-center-baby-and-family-c...,St. Bernardine Medical Center Baby and Family ...,"Family Practicefamilydr, Pediatricianspediatri...",https://www.yelp.com/biz/st-bernardine-medical...,Disregard the fact that I'm a man. My wife was...,"403 E Highland AveSan Bernardino, CA 92404",San Bernardino,CA,92404.0,1,0,34.135776,-117.275612,9.098814e+09
3,beaver-medical-group-highland-2,Beaver Medical Group,"Family Practicefamilydr, Internal Medicineinte...",https://www.yelp.com/biz/beaver-medical-group-...,First time ever going to this particular Beave...,"7223 Church StSte CHighland, CA 92346",Highland,CA,92346.0,1,0,34.122251,-117.173181,9.098621e+09
4,ali-uzma-r-md-fontana-2,Ali Uzma R MD,"Pediatricianspediatricians,",https://www.yelp.com/biz/ali-uzma-r-md-fontana-2,I have been going to Dr. Ali for over 7 years ...,"Fontana, CA 92336",Fontana,CA,92336.0,1,0,34.156970,-117.482160,9.093553e+09


In [43]:
# I was assigned 870 - 1740
base_doctor_url_list = original_doctor_df['URL'][870:1740].tolist()
base_doctor_url_list[0]

'https://www.yelp.com/biz/steven-e-hodgkin-md-victorville'

In [6]:
def random_sleep(min_seconds=60, max_seconds=90):
  sleep_seconds = random.randrange(min_seconds, max_seconds)
  print(f'Sleeping for {sleep_seconds} seconds...')
  time.sleep(sleep_seconds)

In [8]:
import requests
def url_to_soup(url, min_seconds=60, max_seconds=90):
  random_sleep(min_seconds, max_seconds)  # sleep at least a minute between any request
  print(f'Calling url: {url}')
  response = requests.get(url).text  # Make a GET request to the target URL to get the raw HTML data
  soup = BeautifulSoup(response,'html.parser')  # Use BeautifulSoup to parse HTML
  return soup

In [9]:
def add_doctor_without_review(doctors_without_reviews_df, base_doctor_url):
  # rebuild the dataframe, but with the new doctor URL
  doc_list = doctors_without_reviews_df['url'].tolist()
  doc_list.append(base_doctor_url)  
  doctors_without_reviews_df = pd.DataFrame(doc_list, columns=['url'])
  doctors_without_reviews_df.to_pickle('/content/drive/MyDrive/Mydata/doctors_without_reviews_df_PICKLE')
  print('************** NO REVIEWS FOUND **************')

In [11]:
def get_review_count(soup, review_count_class_ids):
  # review count is needed to know how many pages to scrape.  This ID can be one of multiple
  number_reviews = -1
  for review_count_class_id in review_count_class_ids:
    review_soup = soup.find(class_=review_count_class_id)
    if review_soup:      
      number_reviews = review_soup.text
      break;
  
  number_reviews_string_list = re.findall('\d+', number_reviews)
  if number_reviews_string_list:  # make sure the list isn't empty
    number_reviews = int(number_reviews_string_list[0])   # Use rex to extract the numbers from a string 
  else:
    return -1

  print(f'review count: {number_reviews}')
  return number_reviews

In [39]:
# Start with wherever we left off last
review_df = pd.read_pickle('/content/drive/MyDrive/Mydata/review_df_PICKLE')

# Otherwise, use this if starting fresh
#columns = ['base_doctor_url', 'review_username', 'review_rating', 'review_date', 'review']
#review_df = pd.DataFrame(columns=columns)

# Persist the docs without reviews, so they can be skipped without the sleep delay just finding out again that there are no reviews.
doctors_without_reviews_df = pd.read_pickle('/content/drive/MyDrive/Mydata/doctors_without_reviews_df_PICKLE')

In [41]:
len(review_df)

1247

In [ ]:
##### FOR TESTING START WITH 1 doctor. Comment this out when running for real.
#base_doctor_url_list = base_doctor_url_list[:2]
#base_doctor_url_list = base_doctor_url_list[:2]
#base_doctor_url_list = ['https://www.yelp.com/biz/a-doctors-weight-loss-clinic-moreno-valley-2']
#base_doctor_url_list = ['https://www.yelp.com/biz/a-doctors-weight-loss-clinic-moreno-valley-2?start=10']

#doctors_already_done = review_df['base_doctor_url'].unique()
#doctors_already_done = []

# Yelp HTML class IDs
review_count_class_ids = ['css-1h1j0y3', 'css-bq71j2']
review_class_id = 'review__373c0__3MsBX'  # was review__373c0__13kpL
star_class_id = 'i-stars__373c0___sZu0'  # was i-stars__373c0__1T6rz
username_class_id = 'css-166la90'
date_class_id = 'css-e81eai'
content_class_id = 'raw__373c0__tQAx6' # was raw__373c0__3rcx7

count_series = review_df['base_doctor_url'].value_counts()

total_doctors_count = len(base_doctor_url_list)
for index, base_doctor_url in enumerate(base_doctor_url_list): 
  # Check if we already did this one
  # if base_doctor_url in doctors_already_done:
  #   print(f'{index+1} of {total_doctors_count}: {base_doctor_url} - Done')
  #   continue
  if base_doctor_url in doctors_without_reviews_df['url'].values:
    print(f'{index+1} of {total_doctors_count}: {base_doctor_url} - Previously searched - NO REVIEWS')
    continue

  # The ones already in the list, with less than 10 reviews don't need to be attempted again
  #print(type(count_series))
  already_has_10 = False
  if base_doctor_url in count_series:
    previous_review_count = count_series[base_doctor_url]
    print(f'{index+1} of {total_doctors_count}: {base_doctor_url} - Previously searched - {previous_review_count} reviews')
    if previous_review_count < 10:
      continue
    else:
      already_has_10 = True
  else:
    print(f'{index+1} of {total_doctors_count}: {base_doctor_url} - New doc')

  # for the first URL, make sure to handle re-directs, which are common
  time.sleep (random.randint(30, 40))
  res = urllib.request.urlopen(base_doctor_url)
  final_url = res.geturl()
  if final_url != base_doctor_url:
    print(f'Handle redirect to: {final_url}')
    base_doctor_url = final_url

  # Start with souping the initial page with the URL from the input CSV
  first_soup = url_to_soup(base_doctor_url, min_seconds=120, max_seconds=130) # for the first of each doctor, wait at least 2 minutes 

  # From that first page, get the number of reviews
  number_reviews = get_review_count(first_soup, review_count_class_ids)
  # if there are no reviews, then we are done?
  if number_reviews <= 0:
    add_doctor_without_review(doctors_without_reviews_df, base_doctor_url)
    continue

  if already_has_10 and number_reviews == 10:
    print('  ** 10 total reviews. Done')
    continue

  soup_findall_resultsets = []
  if not already_has_10: # don't get the first 10 again if they are already there
    first_set_of_reviews = first_soup.findAll(class_=review_class_id)
    print(f'FIRST SET OF REVIEWS SCRAPED: Count of initial reviews in soup_findall_resultsets: {len(first_set_of_reviews)}')
    soup_findall_resultsets.append(first_set_of_reviews)

  # Create the rest of the URLs needed to get all reviews for this doctor
  subsequent_review_page_urls = []
  for i in range (0, number_reviews, 10):
    subsequent_review_page_urls.append(f'{base_doctor_url}?start={str(i)}')
  del subsequent_review_page_urls[0]  # Get rid of the first one, because adding start=0 is the same as the base url

  # Make the rest of the requests
  for url_to_request in subsequent_review_page_urls:
    next_soup = url_to_soup(url_to_request, min_seconds=60, max_seconds=90) # one minute to 1.5 minute wait
    new_resultset = next_soup.findAll(class_=review_class_id)
    print(f'NEXT SET OF REVIEWS SCRAPED: Count of reviews in new_resultset: {len(new_resultset)}')
    soup_findall_resultsets.append(new_resultset)

  # Combine the soup results into one ResultSet
  soup_resultset = soup_findall_resultsets[0]
  #print(f'First URL count of results in soup_resultset: {len(soup_resultset)}')
  # print(f'Second URL count of results in soup_resultset: {len(soup_resultset[1])}')

  for soup_findall_resultset in soup_findall_resultsets[1:]:
    #print(f'Extend URL count of results in soup_resultset: {len(soup_findall_resultset)}')
    soup_resultset.extend(soup_findall_resultset)
    # soup_resultset_extend_list_one.extend([soup_findall_resultset])
    # soup_resultset_extend_one.extend(soup_findall_resultset)
    print(f'Updated count of results in soup_resultset: {len(soup_resultset)}')

  for review in soup_resultset:
    # extract attribute value using the key aria-label for star ratings. 
    review_rating_string = review.find (class_=star_class_id)['aria-label']
    # \d+ is a regular expression pattern to extract numbers from a string
    review_rating = float (re.findall('\d+', review_rating_string)[0])  

    review_username = review.find (class_=username_class_id).text # the user name for the review
    review_date = review.find (class_=date_class_id).text # the date of the review
    review_content = review.find(class_=content_class_id, attrs={'lang':'en'}).text  # the text of the review

    single_review = {'base_doctor_url': base_doctor_url, 'review_username': review_username,
                     'review_rating': review_rating, 'review_date': review_date, 'review': review_content}
    review_df = review_df.append(single_review, ignore_index = True)
  
  review_df.to_pickle('/content/drive/MyDrive/Mydata/review_df_PICKLE')

print('DONE')

<class 'pandas.core.series.Series'>
1 of 870: https://www.yelp.com/biz/steven-e-hodgkin-md-victorville - Previously searched - 9 reviews
<class 'pandas.core.series.Series'>
2 of 870: https://www.yelp.com/biz/agnes-quion-md-victorville - Previously searched - 2 reviews
<class 'pandas.core.series.Series'>
3 of 870: https://www.yelp.com/biz/hesperia-physical-therapy-and-aquatics-hesperia - Previously searched - 10 reviews
Sleeping for 125 seconds...
Calling url: https://www.yelp.com/biz/hesperia-physical-therapy-and-aquatics-hesperia
review count: 10
  ** 10 total reviews. Done
<class 'pandas.core.series.Series'>
4 of 870: https://www.yelp.com/biz/stephen-chow-md-apple-valley - Previously searched - 7 reviews
<class 'pandas.core.series.Series'>
5 of 870: https://www.yelp.com/biz/stanton-paul-md-apple-valley - Previously searched - 2 reviews
<class 'pandas.core.series.Series'>
6 of 870: https://www.yelp.com/biz/dr-mikes-medical-clinic-apple-valley - Previously searched - 10 reviews
Sleepin

In [26]:
review_df.to_csv('/content/drive/MyDrive/Mydata/review_df.csv')